In [3]:
pip install torch torchvision pillow numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Unzip model file
import zipfile

zip_path = "breakhis_classifier.zip" 
extract_dir = "breakhis_classifier_unzipped"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Model unzipped to:", extract_dir)

✅ Model unzipped to: breakhis_classifier_unzipped


In [8]:
# Verify directory
import os

print("Current working directory:", os.getcwd())
print("Contents:", os.listdir())

📁 Current working directory: /home/sagemaker-user/breakhis_classifier_unzipped
📦 Contents: ['config.json', 'checkpoint-epoch1.pth', 'model_best.pth', 'checkpoint-epoch2.pth', 'checkpoint-epoch3.pth', 'checkpoint-epoch4.pth', 'checkpoint-epoch5.pth', 'checkpoint-epoch6.pth', 'checkpoint-epoch7.pth', 'checkpoint-epoch8.pth', 'checkpoint-epoch9.pth', 'checkpoint-epoch10.pth', 'checkpoint-epoch11.pth', 'checkpoint-epoch12.pth', 'checkpoint-epoch13.pth', 'checkpoint-epoch14.pth', 'checkpoint-epoch15.pth']


In [5]:
# Initialize model
import torch
import torch.nn as nn
from torchvision import models

def densenet121(num_classes=2):
    model = models.densenet121(pretrained=False) 
    model.classifier = nn.Linear(model.classifier.in_features, num_classes)
    return model

In [6]:
import sys
sys.path.append("/home/sagemaker-user/breast_cancer_detection") 

import parse_config 

In [7]:
# Verify directory & files
os.chdir("/home/sagemaker-user/breakhis_classifier_unzipped")

print("Working dir:", os.getcwd())
print("Files here:", os.listdir())

model = densenet121(num_classes=2)

# load checkpoint
checkpoint = torch.load("model_best.pth", map_location="cpu", weights_only=False)
print("Checkpoint keys:", checkpoint.keys())

# load model weights
model.load_state_dict(checkpoint['state_dict'])
model.eval()

Working dir: /home/sagemaker-user/breakhis_classifier_unzipped
Files here: ['config.json', 'checkpoint-epoch1.pth', 'model_best.pth', 'checkpoint-epoch2.pth', 'checkpoint-epoch3.pth', 'checkpoint-epoch4.pth', 'checkpoint-epoch5.pth', 'checkpoint-epoch6.pth', 'checkpoint-epoch7.pth', 'checkpoint-epoch8.pth', 'checkpoint-epoch9.pth', 'checkpoint-epoch10.pth', 'checkpoint-epoch11.pth', 'checkpoint-epoch12.pth', 'checkpoint-epoch13.pth', 'checkpoint-epoch14.pth', 'checkpoint-epoch15.pth']


/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Checkpoint keys: dict_keys(['arch', 'epoch', 'state_dict', 'optimizer', 'monitor_best', 'config'])


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [8]:
# Unzip dataset
os.chdir("/home/sagemaker-user")
abszip_path = "upscaled_generated_images.zip" 
extract_dir = "upscaled_generated_images_unzipped"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("Images unzipped to:", extract_dir)

✅ Images unzipped to: upscaled_generated_images_unzipped


In [9]:
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import ImageFilter

class UnlabeledImageFolder(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_filenames = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_name = self.image_filenames[idx]
        img_path = os.path.join(self.folder_path, img_name)
        image = Image.open(img_path).convert('RGB')
        image = image.filter(ImageFilter.SHARPEN)  # Apply sharpen filter
        if self.transform:
            image = self.transform(image)
        return image, img_name

# Define transforms (same as training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# Create dataset and dataloader
folder_path = "generated_images_unzipped/generated_images"
dataset = UnlabeledImageFolder(folder_path, transform=transform)
loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=False)
print(f"Found {len(dataset)} images in the folder.")

import torch.nn.functional as F

# Predict
model.eval()
predictions = []

with torch.no_grad():
    for images, filenames in loader:
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)  # Probabilities
        _, preds = torch.max(probs, 1)     # Predicted class

        for fname, prob, pred in zip(filenames, probs, preds):
            confidence = prob[pred].item()
            print(f"{fname}: Pred = {pred.item()}, Confidence = {confidence:.4f}")
            predictions.append((fname, pred.item(), confidence))

Found 300 images in the folder.
pathology_gan_img_1.png: Pred = 0, Confidence = 0.6622
pathology_gan_img_10.png: Pred = 0, Confidence = 0.6123
pathology_gan_img_100.png: Pred = 1, Confidence = 0.6284
pathology_gan_img_101.png: Pred = 1, Confidence = 0.9671
pathology_gan_img_102.png: Pred = 1, Confidence = 0.9778
pathology_gan_img_103.png: Pred = 1, Confidence = 0.7321
pathology_gan_img_104.png: Pred = 1, Confidence = 0.9313
pathology_gan_img_105.png: Pred = 1, Confidence = 0.9961
pathology_gan_img_106.png: Pred = 1, Confidence = 0.9893
pathology_gan_img_107.png: Pred = 0, Confidence = 0.5011
pathology_gan_img_108.png: Pred = 1, Confidence = 0.9655
pathology_gan_img_109.png: Pred = 1, Confidence = 0.8796
pathology_gan_img_11.png: Pred = 1, Confidence = 0.7318
pathology_gan_img_110.png: Pred = 1, Confidence = 0.9818
pathology_gan_img_111.png: Pred = 1, Confidence = 0.9885
pathology_gan_img_112.png: Pred = 1, Confidence = 0.9797
pathology_gan_img_113.png: Pred = 1, Confidence = 0.9444
pat